## References

- [Machine learning meets pKa](https://github.com/czodrowskilab/Machine-learning-meets-pKa)
- [MolGpKa](https://github.com/Xundrug/MolGpKa)
- [SMARTS of functional groups](https://github.com/rdkit/rdkit/blob/master/Data/FunctionalGroups.txt)
- [RDKit Fragment Module Document](https://rdkit.org/docs/source/rdkit.Chem.Fragments.html)
- [Introduction to SMILES](https://www.daylight.com/dayhtml/doc/theory/theory.smiles.html)

## Load Data

In [6]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import rdkit.Chem as Chem
import rdkit.Chem.AllChem as AllChem
from rdkit.Chem import MACCSkeys
from rdkit.Chem import PandasTools
import rdkit.Chem.Fragments as Fragments
import torch
from torch_geometric.data import Data
from pysmilesutils.tokenize import SMILESAtomTokenizer

In [2]:
infile = "combined_training_datasets_unique.sdf"
name = os.path.splitext(os.path.basename(infile))[0]

all_df = PandasTools.LoadSDF(infile)
all_df.head()

[23:05:06] Warning: ambiguous stereochemistry - overlapping neighbors  - at atom 13 ignored


,pKa,marvin_pKa,marvin_atom,marvin_pKa_type,original_dataset,ID,ROMol
0,6.21,6.09,10,basic,['chembl25'],1702768,<rdkit.Chem.rdchem.Mol object at 0x7f933163b7d0>
1,7.46,8.2,9,basic,['chembl25'],273537,<rdkit.Chem.rdchem.Mol object at 0x7f92fa2f5310>
2,4.2,3.94,9,basic,['datawarrior'],7175,<rdkit.Chem.rdchem.Mol object at 0x7f92fa2f5380>
3,3.73,5.91,8,acidic,['datawarrior'],998,<rdkit.Chem.rdchem.Mol object at 0x7f92fa2f53f0>
4,11.0,8.94,13,basic,['chembl25'],560562,<rdkit.Chem.rdchem.Mol object at 0x7f92fa2f5460>


In [3]:
print(f"Number of molecules: {all_df.shape[0]}")
pkas = np.array(all_df["pKa"].tolist(), dtype=float)
print(f"Min pKa: {min(pkas)}, Max pKa: {max(pkas)}")

Number of molecules: 5994
Min pKa: 2.0, Max pKa: 11.99


## Functional Group

In [4]:
outfile = f"{name}_training.npy"
patterns = []
for patstr in dir(Chem.Fragments):
    if patstr.startswith("fr"):
        patterns.append(patstr)
print(f"Number of fragment patterns: {len(patterns)}")

PATTERNS = [getattr(Fragments, patstr) for patstr in patterns]

def featurize(mol):
    counts = [pattern(mol) for pattern in PATTERNS]
    return counts

X = []
Y = []

for idx, row in all_df.iterrows():
    x = featurize(row["ROMol"])
    X.append(x)
    Y.append(row["pKa"])
X = np.array(X, dtype=float)
Y = np.array(Y, dtype=float).reshape(-1, 1)

np.save(outfile, np.hstack([X, Y]))

Number of fragment patterns: 85


## Fingerprint

### Morgan Fingerprint

In [22]:
outfile = f"{name}_fp_training.npy"

def featurize(mol):
    # fpgen = AllChem.GetRDKitFPGenerator()
    # fp = np.array(fpgen.GetFingerprint(mol)).tolist()
    fp = np.array(AllChem.GetMorganFingerprintAsBitVect(mol, radius=3, nBits=4096, useFeatures=True))
    return fp

X = []
Y = []

for idx, row in all_df.iterrows():
    x = featurize(row["ROMol"])
    X.append(x)
    Y.append(row["pKa"])
X = np.array(X, dtype=float)
Y = np.array(Y, dtype=float).reshape(-1, 1)

np.save(outfile, np.hstack([X, Y]))

[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerator
[14:16:12] DEPRECATION WARNING: please use MorganGenerat

### MACCS Fingerprint

In [24]:
outfile = f"{name}_maccs_training.npy"

def featurize(mol):
    fp = np.array(MACCSkeys.GenMACCSKeys(mol))
    return fp

X = []
Y = []

for idx, row in all_df.iterrows():
    x = featurize(row["ROMol"])
    X.append(x)
    Y.append(row["pKa"])
X = np.array(X, dtype=float)
Y = np.array(Y, dtype=float).reshape(-1, 1)

np.save(outfile, np.hstack([X, Y]))

## Molecular Graph

In [12]:
TYPES = [
    Chem.rdchem.BondType.SINGLE,
    Chem.rdchem.BondType.DOUBLE,
    Chem.rdchem.BondType.TRIPLE,
    Chem.rdchem.BondType.AROMATIC,
    Chem.rdchem.BondType.OTHER,
    ]
TYPE2DCT = {t: idx for idx, t in enumerate(TYPES)}

ATOMS = ['C', 'O', 'S', 'N', 'P', 'H', 'F', 'Cl', 'Br', 'I', 'UNK']
ATOM2DCT = {ele: np.eye(len(ATOMS))[idx] for idx, ele in enumerate(ATOMS)}

def featurize(mol):
    x = []
    for idx, at in enumerate(mol.GetAtoms()):
        ele = at.GetSymbol()
        ele = ele if ele in ATOMS else 'UNK'
        emd = ATOM2DCT[ele]
        x.append(emd)
    x = torch.from_numpy(np.array(x))
    
    bonds = []
    types = []
    for bond in mol.GetBonds():
        idx1 = bond.GetBeginAtomIdx()
        idx2 = bond.GetEndAtomIdx()
        t = bond.GetBondType()
        t = t if t in TYPES else Chem.rdchem.BondType.OTHER
        bonds.append([idx1, idx2])
        types.append(TYPE2DCT[t])
        bonds.append([idx2, idx1])
        types.append(TYPE2DCT[t])
    edge_index = torch.from_numpy(np.array(bonds))
    edge_type = torch.from_numpy(np.array(types))

    data = Data(x=x, edge_index=edge_index.t().contiguous(), edge_type=edge_type)
    return data

outfile = f"{name}_graph_training.pkl"
DATA = []
for idx, row in all_df.iterrows():
    data = featurize(row["ROMol"])
    y = torch.Tensor([float(row["pKa"])])
    DATA.append((data, y))

with open(outfile, "wb") as file:
    pickle.dump(DATA, file)

## SMILES
- The vocabulary of SMILES is downloaded from [DeepChem vocab](https://github.com/deepchem/deepchem/blob/2.4.0/deepchem/feat/tests/data/vocab.txt)
- The convertion from SMILES to molecule is unique but not vice versa. SMILES augmentation is usually required to learn non-canonical SMILES. As a result, instead of writing SMILES strings to files, we designed a `pytorch..utils.data.Dataset` to sample various SMILES for a single molecule on the fly during training.